In [100]:
import os
from datetime import datetime

import psycopg2

from pyspark.sql import SparkSession
from pyspark.sql import Window
import pyspark.sql.functions as F

from hdfs import InsecureClient

In [2]:
spark = SparkSession\
    .builder\
    .config('spark.driver.extraClassPath'
            , '/home/user/shared_folder/Distrib/postgresql-42.2.23.jar')\
    .master('local')\
    .appName("homework_6")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/22 13:51:01 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/22 13:51:01 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/02/22 13:51:01 WARN util.Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
# Load Pagila tables to Bronze HDFS

In [4]:
pg_creds = {
    'host': '192.168.1.56',
    'port': '5432',
    'database': 'pagila',
    'user': 'pguser',
    'password': 'secret',
}

In [5]:
tables_to_load = (  
     'actor'
    ,'category'
    ,'film'
    ,'film_actor'
    ,'film_category'
    ,'customer'
    ,'address'
    ,'city'
    ,'inventory'
    ,'rental'
)

In [6]:
hdfs_url = 'http://127.0.0.1:50070/'

In [7]:
client_hdfs = InsecureClient(hdfs_url, user='user')

In [8]:
current_date = datetime.now().strftime("%Y-%m-%d")

In [10]:
for table_name in tables_to_load:
    
    bronze_dir = os.path.join('/', 'datalake', 'bronze', 'pagila', table_name, current_date)
                               
    with psycopg2.connect(**pg_creds) as pg_connection:
        cursor = pg_connection.cursor()
        
        with client_hdfs.write(os.path.join(bronze_dir, table_name + '.csv'), overwrite=True) as csv_file:
            cursor.copy_expert(f"COPY {table_name} TO STDOUT WITH HEADER CSV", csv_file)

In [11]:
# Read used csv files to DataFrames

In [54]:
actor_df = spark.read.load(
    os.path.join('/', 'datalake', 'bronze', 'pagila', 'actor', current_date, 'actor.csv')
    , header="true"
    , inferSchema="true"
    , format="csv"
)
category_df = spark.read.load(
    os.path.join('/', 'datalake', 'bronze', 'pagila', 'category', current_date, 'category.csv')
    , header="true"
    , inferSchema="true"
    , format="csv"
)
film_df = spark.read.load(
    os.path.join('/', 'datalake', 'bronze', 'pagila', 'film', current_date, 'film.csv')
    , header="true"
    , inferSchema="true"
    , format="csv"
)
film_actor_df = spark.read.load(
    os.path.join('/', 'datalake', 'bronze', 'pagila', 'film_actor', current_date, 'film_actor.csv')
    , header="true"
    , inferSchema="true"
    , format="csv"
)
film_category_df = spark.read.load(
    os.path.join('/', 'datalake', 'bronze', 'pagila', 'film_category', current_date, 'film_category.csv')
    , header="true"
    , inferSchema="true"
    , format="csv"
)
customer_df = spark.read.load(
    os.path.join('/', 'datalake', 'bronze', 'pagila', 'customer', current_date, 'customer.csv')
    , header="true"
    , inferSchema="true"
    , format="csv"
)
address_df = spark.read.load(
    os.path.join('/', 'datalake', 'bronze', 'pagila', 'address', current_date, 'address.csv')
    , header="true"
    , inferSchema="true"
    , format="csv"
)
city_df = spark.read.load(
    os.path.join('/', 'datalake', 'bronze', 'pagila', 'city', current_date, 'city.csv')
    , header="true"
    , inferSchema="true"
    , format="csv"
)
inventory_df = spark.read.load(
    os.path.join('/', 'datalake', 'bronze', 'pagila', 'inventory', current_date, 'inventory.csv')
    , header="true"
    , inferSchema="true"
    , format="csv"
)
rental_df = spark.read.load(
    os.path.join('/', 'datalake', 'bronze', 'pagila', 'rental', current_date, 'rental.csv')
    , header="true"
    , inferSchema="true"
    , format="csv"
)

In [13]:
#1 вывести количество фильмов в каждой категории, отсортировать по убыванию.

In [55]:
result_1 = film_category_df\
    .join(category_df
         , film_category_df.category_id == category_df.category_id
         , 'inner')\
    .select(category_df.name.alias('category_name'))

result_1 = result_1.groupBy('category_name')\
    .count().orderBy(F.desc('count')).show()

+-------------+-----+
|category_name|count|
+-------------+-----+
|       Sports|   74|
|      Foreign|   73|
|       Family|   69|
|  Documentary|   68|
|    Animation|   66|
|       Action|   64|
|          New|   63|
|        Drama|   62|
|       Sci-Fi|   61|
|        Games|   61|
|     Children|   60|
|       Comedy|   58|
|       Travel|   57|
|     Classics|   57|
|       Horror|   56|
|        Music|   51|
+-------------+-----+



In [18]:
#2 вывести 10 актеров, чьи фильмы большего всего арендовали, отсортировать по убыванию.

In [56]:
result_2 = film_actor_df\
    .join(actor_df
         , film_actor_df.actor_id == actor_df.actor_id
         , 'inner')\
    .select(film_actor_df.film_id, actor_df.first_name, actor_df.last_name)

result_2 = film_df.join(result_2
                      , film_df.film_id == result_2.film_id
                      , 'inner')\
                .select(result_2.first_name, result_2.last_name, film_df.rental_duration)

result_2 = result_2.groupBy(['first_name','last_name'])\
    .agg({'rental_duration': 'sum'})

result_2 = result_2.orderBy(result_2['sum(rental_duration)'].desc()).show(10)

+----------+---------+--------------------+
|first_name|last_name|sum(rental_duration)|
+----------+---------+--------------------+
|     SUSAN|    DAVIS|                 484|
|      GINA|DEGENERES|                 418|
|    WALTER|     TORN|                 402|
|      MARY|   KEITEL|                 384|
|   MATTHEW|   CARREY|                 380|
|   GROUCHO|    DUNST|                 366|
|    ANGELA|   HUDSON|                 366|
|    SANDRA|   KILMER|                 362|
|     HENRY|    BERRY|                 360|
|       UMA|     WOOD|                 358|
+----------+---------+--------------------+
only showing top 10 rows



In [36]:
#3 вывести категорию фильмов, на которую потратили больше всего денег.

In [132]:
result_3 = film_category_df\
    .join(category_df
         , film_category_df.category_id == category_df.category_id
         , 'inner')\
    .join(film_df
         ,  film_category_df.film_id == film_df.film_id
         , 'inner')\
    .select(category_df.name.alias('category_name'), film_df.replacement_cost)\
    .groupBy('category_name')\
    .agg({'replacement_cost': 'sum'})

result_3 = result_3.orderBy(result_3['sum(replacement_cost)'].desc()).show(1)

+-------------+---------------------+
|category_name|sum(replacement_cost)|
+-------------+---------------------+
|       Sports|   3018.5199999999904|
+-------------+---------------------+
only showing top 1 row



In [44]:
#4 вывести названия фильмов, которых нет в inventory.

In [89]:
result_4 = film_df.join(inventory_df
                      , film_df.film_id == inventory_df.film_id
                      , 'left')\
                .select(film_df.film_id, film_df.title, inventory_df.film_id.alias('film_id2'))

result_4.where(F.col('film_id2').isNull()).select('title').distinct().sort('title').show()

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



In [88]:
result_4_1 = film_df.select('film_id').subtract(inventory_df.select('film_id')).show()

+-------+
|film_id|
+-------+
|    148|
|    108|
|    950|
|    642|
|    874|
|    497|
|    332|
|    192|
|    860|
|    128|
|    671|
|    325|
|    386|
|    955|
|    359|
|    419|
|     41|
|    607|
|    318|
|    742|
+-------+
only showing top 20 rows



In [91]:
#5 вывести топ 3 актеров, которые больше всего появлялись в фильмах в категории “Children”. 
# Если у нескольких актеров одинаковое кол-во фильмов, вывести всех..

In [113]:
result_5 = actor_df\
    .join(film_actor_df
         , actor_df.actor_id == film_actor_df.actor_id
         , 'inner')\
    .join(film_df
         , film_actor_df.film_id ==film_df.film_id
         , 'inner')\
    .join(film_category_df
         , film_df.film_id == film_category_df.film_id
         , 'inner')\
     .join(category_df
         , film_category_df.category_id == category_df.category_id
         , 'inner')\
    .where(category_df.name == 'Children')\
    .select(actor_df.first_name, actor_df.last_name)\
    .groupBy(F.col('first_name'), F.col('last_name'))\
    .count()\
    .withColumn('appearance_rank', F.dense_rank().over(Window.orderBy(F.desc('count'))))

result_5 = result_5\
    .where(result_5.appearance_rank <=3)\
    .select('first_name','last_name',F.col('count').alias('appearance')).show()


22/02/22 18:09:27 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+---------+----------+
|first_name|last_name|appearance|
+----------+---------+----------+
|     HELEN|   VOIGHT|        14|
|     SUSAN|    DAVIS|        12|
|     KEVIN|  GARLAND|        10|
|     RALPH|     CRUZ|        10|
|    WHOOPI|     HURT|        10|
|      MARY|    TANDY|        10|
+----------+---------+----------+



In [114]:
#6 вывести города с количеством активных и неактивных клиентов (активный — customer.active = 1). 
# Отсортировать по количеству неактивных клиентов по убыванию.

In [171]:
result_6 = customer_df\
    .join(address_df
          , customer_df.address_id == address_df.address_id
          , 'inner')\
    .join(city_df
          , address_df.city_id == city_df.city_id
          , 'inner')\
    .select(city_df.city,
        F.when(customer_df.active == 1, 1).otherwise(0).alias('isActive'),
        F.when(customer_df.active == 0, 1).otherwise(0).alias('isInactive'))
    
result_6 = result_6\
    .groupBy('city')\
    .agg({'isActive': 'sum', 'isInactive': 'sum'})\
    .orderBy(F.col('sum(isInactive)').desc(), 'city').show()

+------------------+---------------+-------------+
|              city|sum(isInactive)|sum(isActive)|
+------------------+---------------+-------------+
|            Amroha|              1|            0|
|           Bat Yam|              1|            0|
|  Charlotte Amalie|              1|            0|
|     Coatzacoalcos|              1|            0|
|            Daxian|              1|            0|
|            Kamyin|              1|            0|
|            Ktahya|              1|            0|
|        Kumbakonam|              1|            0|
|         Najafabad|              1|            0|
|         Pingxiang|              1|            0|
|   Southend-on-Sea|              1|            0|
|       Szkesfehrvr|              1|            0|
|          Uluberia|              1|            0|
|           Wroclaw|              1|            0|
|          Xiangfan|              1|            0|
|A Corua (La Corua)|              0|            1|
|              Abha|           

In [172]:
#7 вывести категорию фильмов, у которой самое большое кол-во часов суммарной аренды в городах 
# (customer.address_id в этом city), и которые начинаются на букву “a”. 
# То же самое сделать для городов в которых есть символ “-”.

In [195]:
result_7_prep = film_category_df\
    .join(category_df
         , film_category_df.category_id == category_df.category_id
         , 'inner')\
    .join(film_df
         ,  film_category_df.film_id == film_df.film_id
         , 'inner')\
    .join(inventory_df
         , film_df.film_id == inventory_df.film_id
         , 'inner')\
    .join(rental_df
         ,  inventory_df.inventory_id == rental_df.inventory_id
         , 'inner')\
    .join(customer_df
          , rental_df.customer_id == customer_df.customer_id
          , 'inner')\
    .join(address_df
          , customer_df.address_id == address_df.address_id
          , 'inner')\
    .join(city_df
          , address_df.city_id == city_df.city_id
          , 'inner')\
    .filter(city_df.city.like('a%') | city_df.city.like('%-%'))\
    .select(category_df.name.alias('category_name'),
            F.when(city_df.city.like('a%'), film_df.rental_duration).otherwise(0).alias('rental_duration_1'),
            F.when(city_df.city.like('%-%'), film_df.rental_duration).otherwise(0).alias('rental_duration_2'))\
    .groupBy('category_name')\
    .agg({'rental_duration_1': 'sum', 'rental_duration_2': 'sum'})

result_7_1 = result_7_prep\
    .select('category_name', F.col('sum(rental_duration_1)').alias('rental_duration'), F.lit('for "a" - cities'))\
    .orderBy(F.col('sum(rental_duration_1)').desc()).limit(1)

result_7_2 = result_7_prep\
    .select('category_name', F.col('sum(rental_duration_2)').alias('rental_duration'), F.lit('for "-" - cities'))\
    .orderBy(F.col('sum(rental_duration_2)').desc()).limit(1)

result_7 = result_7_1.union(result_7_2).show()

+-------------+---------------+----------------+
|category_name|rental_duration|for "a" - cities|
+-------------+---------------+----------------+
|       Sports|            132|for "a" - cities|
|      Foreign|            550|for "-" - cities|
+-------------+---------------+----------------+

